In [19]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example data: answers of 6 people to n questions
answers = [
    ["Answer 1 by Person 1", "Answer 2 by Person 1", "Answer 3 by Person 1"],
    ["Answer 1 by Person 2", "Answer 2 by Person 2", "Answer 3 by Person 2"],
    ["Answer 1 by Person 3", "Answer 2 by Person 3", "Answer 3 by Person 3"],
    ["Answer 1 by Person 4", "Answer 2 by Person 4", "Answer 3 by Person 4"],
    ["Answer 1 by Person 5", "Answer 2 by Person 5", "Answer 3 by Person 5"],
    ["Answer 1 by Person 6", "Answer 2 by Person 6", "Answer 3 by Person 6"],
]

# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode answers for each person
embeddings = [model.encode(person_answers) for person_answers in answers]

# Compute pairwise cosine similarity between people
# Flatten embeddings for each person into a single vector (e.g., mean pooling)
flattened_embeddings = [np.mean(person_emb, axis=0) for person_emb in embeddings]

# Calculate cosine similarity
similarity_matrix = cosine_similarity(flattened_embeddings)

# Print the similarity matrix
print("Cosine Similarity Matrix:")
print(similarity_matrix)

Cosine Similarity Matrix:
[[1.0000001  0.95898014 0.9408854  0.92412424 0.8999814  0.8735755 ]
 [0.95898014 1.         0.94265205 0.92497253 0.88337606 0.8607444 ]
 [0.9408854  0.94265205 1.0000001  0.92285705 0.8915873  0.87048334]
 [0.92412424 0.92497253 0.92285705 1.0000001  0.8913319  0.8647856 ]
 [0.8999814  0.88337606 0.8915873  0.8913319  1.0000001  0.88523024]
 [0.8735755  0.8607444  0.87048334 0.8647856  0.88523024 1.        ]]


In [20]:
# Compute pairwise cosine similarity for each question
num_questions = len(answers[0])  # Assuming all people answered the same number of questions
question_similarities = []

for q in range(num_questions):
    # Extract embeddings for the current question across all people
    question_embeddings = [embeddings[person][q] for person in range(len(embeddings))]
    # Compute pairwise cosine similarity for the current question
    question_similarity_matrix = cosine_similarity(question_embeddings)
    # Store the average similarity for the current question
    avg_similarity = np.mean(question_similarity_matrix)
    question_similarities.append(avg_similarity)

# Print the average similarity per question
for i, avg_sim in enumerate(question_similarities, start=1):
    print(f"Average similarity for Question {i}: {avg_sim}")

Average similarity for Question 1: 0.9032753705978394
Average similarity for Question 2: 0.918492317199707
Average similarity for Question 3: 0.9319655895233154


In [22]:
# Compute Mean Pairwise Cosine Similarity
# Compute overall inter-person agreement
total_similarity = 0
count = 0

for q in range(num_questions):
    # Extract embeddings for the current question across all people
    question_embeddings = [embeddings[person][q] for person in range(len(embeddings))]
    # Compute pairwise cosine similarity for the current question
    question_similarity_matrix = cosine_similarity(question_embeddings)
    # Sum all pairwise similarities (excluding self-similarity)
    total_similarity += np.sum(question_similarity_matrix) - np.trace(question_similarity_matrix)
    # Count the number of pairwise comparisons
    count += question_similarity_matrix.size - len(question_similarity_matrix)

# Calculate the overall average similarity
overall_agreement = total_similarity / count
print(f"Overall inter-person agreement: {overall_agreement}")

Overall inter-person agreement: 0.9014933109283447


In [23]:
# Generate a Random Baseline
random_embeddings = [np.random.rand(len(answers[0]), embeddings[0][0].shape[0]) for _ in range(len(answers))]
random_total_similarity = 0

for q in range(num_questions):
    # Extract random embeddings for the current question
    random_question_embeddings = [random_embeddings[person][q] for person in range(len(random_embeddings))]
    # Compute pairwise cosine similarity for the random embeddings
    random_similarity_matrix = cosine_similarity(random_question_embeddings)
    # Sum all pairwise similarities (excluding self-similarity)
    random_total_similarity += np.sum(random_similarity_matrix) - np.trace(random_similarity_matrix)

# Calculate the mean pairwise cosine similarity for the random baseline
random_mean_pairwise_similarity = random_total_similarity / count
print(f"Random Baseline Mean Pairwise Cosine Similarity: {random_mean_pairwise_similarity}")

Random Baseline Mean Pairwise Cosine Similarity: 0.7431037437031196
